In [10]:
from sageresearch.discontinuousgalerkin import basis
from sageresearch.discontinuousgalerkin import canonical_element

# Discontinous Galerkin Formulation

## Flux Integral

The dg formulation involves a integral of the flux multiplied by the gradient of the test functions or basis
\dintt{K_i}

In [12]:
basis_1d = basis.get_legendre_basis_1d(4)

In [15]:
type(basis_1d[0])

<class 'sage.symbolic.expression.Expression'>

In [16]:
xi = SR.symbol('xi')

In [18]:
type(basis_1d)

<class 'list'>

In [9]:
type(c[0, 0])

<class 'sage.rings.integer.Integer'>

In [44]:
vertex_list = [[random() for i in range(2)] for j in range(3)]
sorted_vertex_list = vertex_list.copy()
sorted_vertex_list.sort()
print(vertex_list)
print(sorted_vertex_list)

[[0.8360601781029829, 0.9060375342429287], [0.043661581723369536, 0.6839988769620446], [0.8175803372477582, 0.9178723239164148]]
[[0.043661581723369536, 0.6839988769620446], [0.8175803372477582, 0.9178723239164148], [0.8360601781029829, 0.9060375342429287]]


In [37]:
sort(vertex_list)

NameError: name 'sort' is not defined

In [30]:
vertex_list.sort()

In [31]:
vertex_list

[[0.07064579035734775, 0.01593357296003084],
 [0.16634781760903128, 0.4538053686083695],
 [0.2727161948976571, 0.07501490144145284]]